In [ ]:
pip install pg8000

In [ ]:
import os
import ssl
import pg8000
import re

In [ ]:
# conexion a RDS usando el bundle de Amazon, esto no es necesario desde lambda
RDS_CA_PATH = os.getenv(
    "RDS_SSL_CA",
    r"K:\documentos\UNGS\Proyecto Profesional I\Tp principal 2-2025\AlimentAPP\global-bundle.pem",
)

ssl_context = ssl.create_default_context(cafile=RDS_CA_PATH)

In [ ]:
# Variables
ENV = 'dev'

In [ ]:
PASSWORD_INPUT = input('ingrese contraseña')
def get_connection():
    """
    Abre una conexion pg8000 con SSL.
    """
    return pg8000.connect(
        host='db-alimentapp.cm7oyoiiqchb.us-east-1.rds.amazonaws.com',
        port=5432,
        database='postgres',
        user='labo_team',
        password=PASSWORD_INPUT,
        ssl_context=ssl_context,
        timeout=10,
    )

In [ ]:
def run_query(sql: str):
    """
    Ejecuta un SELECT y devuelve filas como diccionarios.
    """
    conn = get_connection()
    try:
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        if not rows:
            return []
        columns = [desc[0] for desc in cur.description]
        return [dict(zip(columns, row)) for row in rows]
    finally:
        conn.close()

def run_command(sql: str):
    """
    Ejecuta INSERT/UPDATE/DELETE y hace commit.
    """
    conn = get_connection()
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    finally:
        conn.close()

# get_clients

In [ ]:
get_clients_query = f'SELECT * FROM {ENV}.cliente'

# Obtengo todos los clientes
clients = run_query(get_clients_query)

# Post nuevo cliente

In [ ]:
class ValidationError(Exception):
    pass


NAME_REGEX = re.compile(r"^[A-Za-zÁÉÍÓÚÑáéíóúñ\s']+$")
PHONE_REGEX = re.compile(r"^[0-9+\-\s()]+$")
EMAIL_REGEX = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
CUIL_REGEX = re.compile(r"^\d{2}-\d{8}-\d$")  # ajustalo si aceptás otro formato

def validate_client_payload(payload: dict[str, str]) -> dict[str, str]:
    """
    Valida que todos los campos del cliente estén presentes y en formato correcto.
    """
    required = [
        "razon_social",
        "email",
        "cuil",
        "nombre_contacto",
        "apellido_contacto",
        "telefono",
    ]
    missing = [key for key in required if not payload.get(key)]
    if missing:
        raise ValidationError(f"Faltan campos obligatorios: {', '.join(missing)}")

    email = payload["email"].strip()
    if not EMAIL_REGEX.fullmatch(email):
        raise ValidationError("email no tiene un formato válido.")

    cuil = payload["cuil"].strip()
    if not CUIL_REGEX.fullmatch(cuil):
        raise ValidationError("cuil no respeta el formato ##-########-#.")

    nombre = payload["nombre_contacto"].strip()
    if not NAME_REGEX.fullmatch(nombre):
        raise ValidationError("nombre_contacto solo admite letras y espacios.")

    apellido = payload["apellido_contacto"].strip()
    if not NAME_REGEX.fullmatch(apellido):
        raise ValidationError("apellido_contacto solo admite letras y espacios.")

    telefono = payload["telefono"].strip()
    if not PHONE_REGEX.fullmatch(telefono):
        raise ValidationError("telefono solo admite dígitos, +, -, () y espacios.")

    return {
        "razon_social": payload["razon_social"].strip(),
        "email": email,
        "cuil": cuil,
        "nombre_contacto": nombre,
        "apellido_contacto": apellido,
        "telefono": telefono,
    }

In [ ]:
input_payload = {
  "razon_social": "Panificados SRL",
  "email": "abc@hotmail.com",
  "cuil": "30-23456728-9",
  "nombre_contacto": "Ana",
  "apellido_contacto": "Gimenez",
  "telefono": "1155556677"
}

In [ ]:
try:
    data = validate_client_payload(input_payload)
except ValidationError as exc:
    #return {"statusCode": 400, "body": json.dumps({"message": str(exc)})}
    print({"statusCode": 400, "body": json.dumps({"message": str(exc)})})

In [ ]:

run_command(
    f"""
    INSERT INTO {ENV}.cliente
        (razon_social, email, cuil, nombre_contacto, apellido_contacto, telefono)
    VALUES
        ('{data["razon_social"]}', '{data["email"]}', '{data["cuil"]}',
         '{data["nombre_contacto"]}', '{data["apellido_contacto"]}', '{data["telefono"]}');
    """
)

# get orden venta

In [ ]:
input_payload = {
  "id": "3",
}

In [ ]:
get_orden_venta_query = f"select * from {ENV}.orden_venta limit 20;"
pedidos = run_query(get_orden_venta_query)
print(pedidos)

# get orden venta by id vendedor

In [ ]:
get_id_empleado = int(input_payload["id"])
get_pedido_query_vendedor = f"select * from {ENV}.orden_venta where id_empleado = {get_id_empleado};"
pedido = run_query(get_pedido_query_vendedor)
print(pedidos)

# get orden venta by id cliente

In [ ]:
input_payload = {
  "id": "lucas"
}

try:
    get_id_pedido = int(input_payload["id"])
except ValueError as exc:
    print({
        "statusCode": 400,
        "body": json.dumps({"message": f"ID inválido: {exc}"})
    })

get_id_cliente = int(input_payload["id"])
get_pedido_query_cliente = f"select * from {ENV}.orden_venta where id_cliente = {get_id_cliente};"
pedido = run_query(get_pedido_query_cliente)
print(pedidos)

# get orden venta by id pedido

In [ ]:
input_payload = {
  "id": "lucas"
}

try:
    get_id_pedido = int(input_payload["id"])
except ValueError as exc:
    print({
        "statusCode": 400,
        "body": json.dumps({"message": f"ID inválido: {exc}"})
    })

get_pedido_query_pedido = f"select * from {ENV}.orden_venta where id = {get_id_pedido};"
pedido = run_query(get_pedido_query_pedido)
print(pedido)

# get orden venta by estado

In [ ]:
input_payload2 = {
  "estado": "pendiente",
}
estados_orden_venta = [
    'pendiente',
    'confirmada',
    'cancelada',
    'en_produccion',
    'lista',
    'entregada'
]

get_estado = str(input_payload2["estado"])

if get_estado not in estados_orden_venta:
    raise ValidationError(f"El estado '{get_estado}' NO existe")
else: 
    get_pedido_query_estado = f"select * from {ENV}.orden_venta where estado = '{get_estado}';"
    pedido = run_query(get_pedido_query_estado)
    print(pedido)


# get productos (select id, nombre from producto)

In [ ]:
get_productos_query = f'SELECT id,nombre FROM {ENV}.producto'

# Obtengo todos los productos
productos = run_query(get_productos_query)
print(productos)

# get ordenes_produccion limit 20

In [ ]:
get_ordenes_produccion_limit20_query = f'SELECT * FROM {ENV}.orden_produccion ORDER BY fecha_creacion DESC LIMIT 20'

# Obtengo todos las ultimas 20 ordenes de produccion
ordenes_produccion = run_query(get_ordenes_produccion_limit20_query)
print(ordenes_produccion)